# 文本预处理

In [8]:
import collections
import re
from d2l import torch as d2l


将数据集读取到由文本行组成的列表中

In [9]:
d2l.DATA_HUB['time_machine'] = (d2l.DATA_URL + 'timemachine.txt',
                                '090b5e7e70c295757f55df93cb0a180b9691891a')
def read_time_machine():  
    """Load the time machine dataset into a list of text lines."""
    with open(d2l.download('time_machine'), 'r') as f:
        lines = f.readlines()
    # 26个字母
    #小写
    #暴力预处理
    return [re.sub('[^A-Za-z]+', ' ', line).strip().lower() for line in lines]

lines = read_time_machine()

print(lines[0])
print(lines[10])

the time machine by h g wells
twinkled and his usually pale face was flushed and animated the


每个文本序列被拆分成一个标记列表

In [11]:
def tokenize(lines, token='word'):  
    """将文本行拆分为单词或字符标记。"""
    #词作为token
    if token == 'word':
        return [line.split() for line in lines]
    #字符作为token
    elif token == 'char':
        return [list(line) for line in lines]
    else:
        print('错误：未知令牌类型：' + token)

tokens = tokenize(lines)
for i in range(11):
    print(tokens[i])

['the', 'time', 'machine', 'by', 'h', 'g', 'wells']
[]
[]
[]
[]
['i']
[]
[]
['the', 'time', 'traveller', 'for', 'so', 'it', 'will', 'be', 'convenient', 'to', 'speak', 'of', 'him']
['was', 'expounding', 'a', 'recondite', 'matter', 'to', 'us', 'his', 'grey', 'eyes', 'shone', 'and']
['twinkled', 'and', 'his', 'usually', 'pale', 'face', 'was', 'flushed', 'and', 'animated', 'the']


构建一个字典，通常也叫做词表（vocabulary），用来将字符串标记映射到从0开始的数字索引中

In [13]:
class Vocab:  
    """文本词表"""
    #把词汇映射到  --- >  带索引的词表
    # min_freq  : 如果word出现的times < min_freq  ,就丢弃
    #reserved_token:开始、结束的token
    def __init__(self, tokens=None, min_freq=0, reserved_tokens=None):
        if tokens is None:
            tokens = []
        if reserved_tokens is None:
            reserved_tokens = []
        counter = count_corpus(tokens)
        # 给词频排序：从大到小
        # 做排序，访问对cache有好处
        self.token_freqs = sorted(counter.items(), key=lambda x: x[1],reverse=True)
        # unk下标 为0
        #<unk> token 是 unique token
        # reserved token 也算 unique token
        self.unk, uniq_tokens = 0, ['<unk>'] + reserved_tokens
        #把token遍历一遍，把freq < min_freq 的丢掉
        # 把token中不在unique token的token放入 unique token
        uniq_tokens += [
            token for token, freq in self.token_freqs
            if freq >= min_freq and token not in uniq_tokens]
        # list: [idx--> token]
        # dict：{token  --->  index}
        self.idx_to_token, self.token_to_idx = [], dict()
        for token in uniq_tokens:
            # 按序列插入 token 
            self.idx_to_token.append(token)
            #序列为0-len-1
            self.token_to_idx[token] = len(self.idx_to_token) - 1
    #unique token的个数
    def __len__(self):
        return len(self.idx_to_token)
    # 传入token  --> 得到token的idex
    def __getitem__(self, tokens):
        #<unk>token 返回0下标
        # tokens 返回的 不是 元组 或者 list --- > ukn
        if not isinstance(tokens, (list, tuple)):
            return self.token_to_idx.get(tokens, self.unk)
        # 根据token返回 idx
        return [self.__getitem__(token) for token in tokens]
    
    # 根据 index 返回 token
    def to_tokens(self, indices):
        #传入 index = 0 
        if not isinstance(indices, (list, tuple)):
            return self.idx_to_token[indices]
        #返回index --> token 
        return [self.idx_to_token[index] for index in indices]

def count_corpus(tokens):  
    """统计标记的频率。"""
    if len(tokens) == 0 or isinstance(tokens[0], list):
        tokens = [token for line in tokens for token in line]
    # 和map的作用一样：统计词出现的频率
    return collections.Counter(tokens)

构建词汇表

In [14]:
vocab = Vocab(tokens)
print(list(vocab.token_to_idx.items())[:10])

[('<unk>', 0), ('the', 1), ('i', 2), ('and', 3), ('of', 4), ('a', 5), ('to', 6), ('was', 7), ('in', 8), ('that', 9)]


将每一行文本转换成一个数字索引列表

In [16]:
for i in [0, 10]:
    print('words:', tokens[i])
    print('indices:', vocab[tokens[i]])

words: ['the', 'time', 'machine', 'by', 'h', 'g', 'wells']
indices: [1, 19, 50, 40, 2183, 2184, 400]
words: ['twinkled', 'and', 'his', 'usually', 'pale', 'face', 'was', 'flushed', 'and', 'animated', 'the']
indices: [2186, 3, 25, 1044, 362, 113, 7, 1421, 3, 1045, 1]


将所有内容打包到load_corpus_time_machine函数中

In [17]:
def load_corpus_time_machine(max_tokens=-1):  
    """返回时光机器数据集的标记索引列表和词汇表。"""
    #获取文本
    lines = read_time_machine()
    #按char做token，26个小写字母，unk，blank
    tokens = tokenize(lines, 'char')
    vocab = Vocab(tokens)
    # print(list(vocab.token_to_idx.items())[:10])
    #传进token得到index
    #集合text所有的index -- > 放到corpus里面
    corpus = [vocab[token] for line in tokens for token in line]
    if max_tokens > 0:
        corpus = corpus[:max_tokens]
    return corpus, vocab

corpus, vocab = load_corpus_time_machine()
len(corpus), len(vocab)

[('<unk>', 0), (' ', 1), ('e', 2), ('t', 3), ('a', 4), ('i', 5), ('n', 6), ('o', 7), ('s', 8), ('h', 9)]


(170580, 28)